In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]

In [ ]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
seq_length = 40
step_size = 3

In [ ]:
sentences = []
next_cahracters = []

for i in range(0, len(text) - seq_length, step_size):
  sentences.append(text[i: i+seq_length])
  next_cahracters.append(text[i+seq_length])

In [ ]:
x = np.zeros((len(sentences), seq_length, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype = np.bool)

<ipython-input-16-1ce032c4932b>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(characters)), dtype = np.bool)


In [ ]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    x[i, t, char_to_index[character]] = 1
  y[i, char_to_index[next_cahracters[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape= (seq_length, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy' , optimizer=RMSprop(learning_rate=0.01))
model.fit(x, y, batch_size=256, epochs = 4)
model.save('textgenerator.model')

Epoch 1/4
651/651 [==============================] - 125s 189ms/step - loss: 2.1724
Epoch 2/4
651/651 [==============================] - 118s 181ms/step - loss: 1.7290
Epoch 3/4
651/651 [==============================] - 118s 181ms/step - loss: 1.6046
Epoch 4/4
651/651 [==============================] - 118s 181ms/step - loss: 1.5337


In [ ]:
model = tf.keras.models.load_model('/content/textgenerator.model')

In [ ]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text)-seq_length-1)
  generated = ''
  sentence = text[start_index: start_index + seq_length]
  generated += sentence
  for i in range(length):
    x = np.zeros((1, seq_length, len(characters)))
    for t, character in enumerate(sentence):
      x[0, t, char_to_index[character]] = 1

    predictions = model.predict(x, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character
  return generated

In [ ]:
print(generate_text(300, 0.1))
print(generate_text(300, 0.2))

d:
why, then 'tis time to arm and give death,
and thou wast the care and the care and the hands
the care and the worse and the care and the hands
the care and the bear the earth the hands.

clown:
i will the earth the hands and the fierd
the care and the breast the care and the worsent with the hands
and what show the hand and the care th
 shall intend to do,
by heaven, i will the cheek the care and still me.

polixenes:
i have a treess of the bear the hands thee,
which see his his hands and the cheer and the care
the hand the earth of the care and be some
should not the care i shall be not the chark.

warwick:
which see him his hands the care and so still him shall
with t
